In [1]:
import os
import torch
from Models.MultiViewViT import MultiViewViT
from load_data import IMG_Folder
import torch.nn as nn

In [2]:
def weights_init(w):
    classname = w.__class__.__name__
    if classname.find('Conv') != -1:
        if hasattr(w, 'weight'):
            # nn.init.kaiming_normal_(w.weight, mode='fan_out', nonlinearity='relu')
            nn.init.kaiming_normal_(w.weight, mode='fan_in', nonlinearity='leaky_relu')
        if hasattr(w, 'bias') and w.bias is not None:
                nn.init.constant_(w.bias, 0)
    if classname.find('Linear') != -1:
        if hasattr(w, 'weight'):
            torch.nn.init.xavier_normal_(w.weight)
        if hasattr(w, 'bias') and w.bias is not None:
            nn.init.constant_(w.bias, 0)
    if classname.find('BatchNorm') != -1:
        if hasattr(w, 'weight') and w.weight is not None:
            nn.init.constant_(w.weight, 1)
        if hasattr(w, 'bias') and w.bias is not None:
            nn.init.constant_(w.bias, 0)

In [3]:
# Load model
model = MultiViewViT(
    image_sizes=[(91, 109), (91, 91), (109, 91)],
    patch_sizes=[(7, 7), (7, 7), (7, 7)],
    num_channals=[91, 109, 91],
    vit_args={
        'emb_dim': 768, 'mlp_dim': 3072, 'num_heads': 12,
        'num_layers': 12, 'num_classes': 1,
        'dropout_rate': 0.1, 'attn_dropout_rate': 0.0
    },
    mlp_dims=[3, 128, 256, 512, 1024, 512, 256, 128, 1]
)
model.apply(weights_init)
model = model.to("cpu")

# Load checkpoint
CheckpointPath = r'C:\Users\Rishabh\training_output_metricsMulti_VIT_best_model.pth.tar'
checkpoint = torch.load(CheckpointPath, map_location="cpu")
state_dict = checkpoint["state_dict"]
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)


<All keys matched successfully>

In [4]:
attentions = []
attention_gradients = []
def get_attention(module, input, output):
    attentions.append(output.cpu())

def get_attention_gradient(module, grad_input, grad_output):
    print('hii')
    # attention_gradients.append(grad_input[0].cpu())
    print('grad_input:- ', grad_input)
    print('grad_output:- ', grad_output)
    attention_gradients.append(grad_input[0])

In [11]:
attention_gradients

[]

In [6]:
model.vit_1.transformer.encoder_layers[0].attn.register_full_backward_hook(get_attention_gradient)

In [7]:
model.vit_1.transformer.encoder_layers[0].attn

SelfAttention(
  (query): LinearGeneral()
  (key): LinearGeneral()
  (value): LinearGeneral()
  (out): LinearGeneral()
)

In [8]:
CheckpointPath = r'C:\Users\Rishabh\trainingMulti_VIT_best_model.pth.tar'
CSVPath = r'C:\Users\Rishabh\Documents\TransBTS\IXI.xlsx'
DataFolder = r'C:\Users\Rishabh\Documents\TrimeseData'
test_data = IMG_Folder(CSVPath, DataFolder)
device = "cpu"

In [9]:
valid_loader = torch.utils.data.DataLoader(test_data
                                         ,batch_size=1
                                         ,num_workers=0
                                         ,pin_memory=True
                                         ,drop_last=True
                                         )

In [3]:
import numpy as np
import cv2
out, targ, ID, Attn1, Attn2, Attn3 = [], [], [], [], [], []
target_numpy, predicted_numpy, ID_numpy = [], [], []
model.eval()
with torch.no_grad():
    for _, (input, ids ,target,male) in enumerate(valid_loader):
        # print('ids:- ', ids)
        # print('input:- ', input.shape)
        # print('target:- ', target)
        # print('male:- ', male)
        
        input = input.to(device).type(torch.FloatTensor)
        
        # ======= convert male lable to one hot type ======= #
        # male = torch.unsqueeze(male,1)
        # male = torch.zeros(male.shape[0],2).scatter_(1,male,1)
        # male = male.type(torch.FloatTensor).to(device)

        target = torch.from_numpy(np.expand_dims(target,axis=1))
        target = target.type(torch.FloatTensor).to(device)

        # ======= compute output and loss ======= #
        output, (attn1, attn2, attn3) = model(input, return_attention_weights=True)
        out.append(output.cpu().numpy())
        targ.append(target.cpu().numpy())
        ID.append(ids)
        attn1= torch.stack(attn1, dim=0)
        attn2 = torch.stack(attn2, dim=0)
        attn3 = torch.stack(attn3, dim=0)
        print('attn1 shape:- ', attn1.shape)
        print('attn2 shape:- ', attn2.shape)
        print('attn3 shape:- ', attn3.shape)
        attn1_mean = attn1.mean(dim=0)
        attn2_mean = attn2.mean(dim=0)
        attn3_mean = attn3.mean(dim=0)
        avg_attn1 = attn1_mean.mean(dim=1)
        avg_attn2 = attn2_mean.mean(dim=1)
        avg_attn3 = attn3_mean.mean(dim=1)
        avg_attn1 = avg_attn1.mean(dim=0)
        avg_attn2 = avg_attn2.mean(dim=0)
        avg_attn3 = avg_attn3.mean(dim=0)
        Attn1.append(avg_attn1)
        Attn2.append(avg_attn2)
        Attn3.append(avg_attn3)


NameError: name 'model' is not defined

In [1]:
Attn1[0].shape, len(Attn1)

NameError: name 'Attn1' is not defined

In [2]:
Attn1 = torch.mean(torch.stack(Attn1), dim=0)
Attn2 = torch.mean(torch.stack(Attn2), dim=0)
Attn3 = torch.mean(torch.stack(Attn3), dim=0)

NameError: name 'torch' is not defined

In [ ]:
Attn1.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Create a dummy 513x513 attention matrix
# In practice, replace this with your actual attention weights
attention_matrix = Attn1

# Plot the attention matrix
plt.figure(figsize=(8, 8))
# plt.imshow(attention_matrix, cmap='viridis', interpolation='nearest')
plt.imshow(attention_matrix, cmap='hot')

plt.colorbar(label="Attention Weight")
plt.title("Attention Matrix (513 x 513)")
plt.xlabel("Key Positions")
plt.ylabel("Query Positions")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Create a dummy 513x513 attention matrix
# In practice, replace this with your actual attention weights
attention_matrix = Attn2

# Plot the attention matrix
plt.figure(figsize=(8, 8))
# plt.imshow(attention_matrix, cmap='viridis', interpolation='nearest')
plt.imshow(attention_matrix, cmap='hot')

plt.colorbar(label="Attention Weight")
plt.title("Attention Matrix (513 x 513)")
plt.xlabel("Key Positions")
plt.ylabel("Query Positions")
plt.tight_layout()
plt.show()


In [ ]:
attention_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Create a dummy 513x513 attention matrix
# In practice, replace this with your actual attention weights
attention_matrix = Attn3

# Plot the attention matrix
plt.figure(figsize=(16, 16))
# plt.imshow(attention_matrix, cmap='viridis', interpolation='nearest')
plt.imshow(attention_matrix, cmap='hot')

plt.colorbar(label="Attention Weight")
plt.title("Attention Matrix (513 x 513)")
plt.xlabel("Key Positions")
plt.ylabel("Query Positions")
plt.tight_layout()
plt.show()

In [ ]:
att_mat = Attn3
residual_att = torch.eye(att_mat.size(1)).to(device="cpu")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Create a dummy 513x513 attention matrix
# In practice, replace this with your actual attention weights
attention_matrix = residual_att

# Plot the attention matrix
plt.figure(figsize=(32, 32))
# plt.imshow(attention_matrix, cmap='viridis', interpolation='nearest')
plt.imshow(attention_matrix, cmap='hot')

plt.colorbar(label="Attention Weight")
plt.title("Attention Matrix (513 x 513)")
plt.xlabel("Key Positions")
plt.ylabel("Query Positions")
plt.tight_layout()
plt.show()

In [ ]:
aug_att_mat = att_mat + residual_att

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Create a dummy 513x513 attention matrix
# In practice, replace this with your actual attention weights
attention_matrix = aug_att_mat

# Plot the attention matrix
plt.figure(figsize=(32, 32))
# plt.imshow(attention_matrix, cmap='viridis', interpolation='nearest')
plt.imshow(attention_matrix, cmap='hot')

plt.colorbar(label="Attention Weight")
plt.title("Attention Matrix (513 x 513)")
plt.xlabel("Key Positions")
plt.ylabel("Query Positions")
plt.tight_layout()
plt.show()

In [ ]:
aug_att_mat

In [ ]:
aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)

In [ ]:
aug_att_mat.shape

In [ ]:
aug_att_mat

In [ ]:
aug_att_mat.sum(dim=-1).unsqueeze(-1).shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example: Create a dummy 513x513 attention matrix
# In practice, replace this with your actual attention weights
attention_matrix = aug_att_mat

# Plot the attention matrix
plt.figure(figsize=(32, 32))
# plt.imshow(attention_matrix, cmap='viridis', interpolation='nearest')
plt.imshow(attention_matrix, cmap='hot')

plt.colorbar(label="Attention Weight")
plt.title("Attention Matrix (513 x 513)")
plt.xlabel("Key Positions")
plt.ylabel("Query Positions")
plt.tight_layout()
plt.show()

In [ ]:
joint_attentions = torch.zeros(aug_att_mat.size()).to(device="cpu")
joint_attentions[0] = aug_att_mat[0]
for n in range(1, aug_att_mat.size(0)):
    joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n - 1])

In [ ]:
joint_attentions.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors

# Example: Create a dummy 513x513 attention matrix
# In practice, replace this with your actual attention weights
attention_matrix = joint_attentions

# Plot the attention matrix
plt.figure(figsize=(32, 32))
plt.imshow(attention_matrix, cmap='viridis', interpolation='nearest')
# plt.imshow(attention_matrix, cmap='hot', interpolation='nearest', aspect='equal')
# plt.imshow(attention_matrix, cmap='hot', interpolation='nearest',
#            norm=mcolors.LogNorm(vmin=attention_matrix.min() + 1e-8, vmax=attention_matrix.max()))
plt.colorbar(label="Attention Weight")
plt.title("Attention Matrix (513 x 513)")
plt.xlabel("Key Positions")
plt.ylabel("Query Positions")
plt.tight_layout()
plt.show()

In [ ]:
joint_attentions.shape

In [ ]:
joint_attentions.sum(dim=[0])

In [ ]:
joint_attentions.sum(dim=[1])

In [ ]:
joint_attentions.sum(dim=[0]) + joint_attentions.sum(dim=[1])

In [ ]:
joint_attentions